In [3]:
import pandas as pd
data = pd.read_csv('train.csv')
data

,text,intent
0,Оформи профиль компании,create_company_profile
1,Сделай регистрацию фирмы онлайн,create_company_profile
2,Добавь профиль юрлица,create_company_profile
3,с нуля добавь данные новой компании,create_company_profile
4,Хочу создать аккаунт компании повторно,create_company_profile
...,...,...
1395,Добавь новую оферту,manage_offer
1396,Сделай публикацию оферты на сегодня,manage_offer
1397,Сделай редактирование и публикацию оферты авто...,manage_offer
1398,онлайн сделай публикацию оферты,manage_offer


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1. Загрузка данных
data = pd.read_csv('train.csv')
X = data['text']
y = data['intent']

# 2. Преобразование текста в числовые признаки
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_vectorized = vectorizer.fit_transform(X)

# 3. Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.42, random_state=82
)

# 4. Обучение модели CatBoost
model = CatBoostClassifier(iterations=100, verbose=10)
model.fit(X_train, y_train)

# 5. Оценка модели
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Learning rate set to 0.5
0:	learn: 1.3453172	total: 168ms	remaining: 16.6s
10:	learn: 0.4807276	total: 318ms	remaining: 2.57s
20:	learn: 0.2395672	total: 467ms	remaining: 1.75s
30:	learn: 0.1600164	total: 615ms	remaining: 1.37s
40:	learn: 0.1181154	total: 769ms	remaining: 1.11s
50:	learn: 0.0894099	total: 961ms	remaining: 923ms
60:	learn: 0.0725689	total: 1.12s	remaining: 719ms
70:	learn: 0.0631680	total: 1.29s	remaining: 526ms
80:	learn: 0.0515109	total: 1.45s	remaining: 339ms
90:	learn: 0.0426426	total: 1.6s	remaining: 159ms
99:	learn: 0.0373933	total: 1.74s	remaining: 0us
Accuracy: 0.9948979591836735
                           precision    recall  f1-score   support

                  add_ecp       1.00      1.00      1.00        93
   create_company_profile       0.96      1.00      0.98        73
          create_contract       1.00      0.99      0.99        88
create_direct_procurement       1.00      0.99      0.99        85
 create_quotation_session       1.00      1.00      1

In [ ]:
test_phrase = ["как создать кс?"]
test_vector = vectorizer.transform(test_phrase)
predicted_class = model.predict(test_vector)
print(f'Предсказанный класс: {predicted_class[0]}')

In [ ]:
model.save_model("catboost_model.cbm")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

In [ ]:
data["intent"].unique()

In [ ]:
model = CatBoostClassifier()
model.load_model("catboost_model.cbm")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Тестовый пример
sample_texts = [
    "Создать новую компанию",
    "Удалить оферту из корзины"
]

# Преобразование текста и предсказание
X_sample = vectorizer.transform(sample_texts)
preds = model.predict(X_sample)

for text, pred in zip(sample_texts, preds):
    print(f"Текст: {text} → Предсказанный интент: {pred}")